* spin lock 스핀락은 lock()안에서 계속 돌게 된다. 
> 프로세스가 사용하려는 lock을 타 프로세스 사용 시 명령어 루프를 실행하며 대기하는 locking 매커니즘

    - Lock을 얻을 수 없다면, 계속 해서 Lock을 확인하며 얻을 때까지 기다린다는 것. Lock이 곧 사용 가능해질 경우 Context Switching을 줄여 cpu의 부담을 덜어준다. 하지만 만약 어떤 스레드가 lock을 오랫동안 유지한다면 오히려 cpu시간을 많이 소모할 가능성이 있다. (비효율적)
    

* sleep lock 슬립락의 경우 큐를 이용한다. 
    - 비교적 장시간 락을 유지한다. 슬립락은 즉각 cpu를 양보한다. 제어권을 넘겨주고는 wait queue로 간다. unlock()에 wakeup 코드가 들어가있다
    
    - 슬립락의 경우 뮤텍스를 떠올릴 수 있다. Mutual Exclusion( 상호 배타)로 하나가 임계영역(Critical Section)에 들어가면 다른 하나는 들어가지 못한다. 
    
    - 하지만 그렇다고 언제나 sleep lock이 유리한 것은 아니다.. cpu 할당해주는 기능이나 코드가 길 수도 있기 때문이다
    
* 임계영역(Critical Section) 코드가 짧으면 spin lock이 유리, 길면 sleep lock이 유리하다

###  세마포어

세마포어는 뮤텍스보다 더 일반적으로 사용되는 개념
* 세마포어 값은 곧 한 번에 자원에 접근할 수 있는 스레드의 수다. 
    - 세마포어 연산에서 남아있는 자원의 수는 곧 공유변수를 의미한다.
    - 세마포어는 threading 모듈을 import 해야 한다.
    
    
* P 연산 
    - P는 임계영역으로 진입하다 라는 뜻으로 임계영역으로 들어가면서 자원수(공유변수)는 감소하는 연산이다
    
* V연산 : P연산과 반대로 임계영역에서 나오면서 자원수(공유변수)는 증가시키는 연산이다

* 스레드 풀 
    - 스레드 풀은 스레드를 미리 여러개 만들어 놓고 worker thread로 대기타게 해두는 것이다
    
* 데드락 (dead lock) : 생각하는 철학자
    - 락을 소팅된 순서로 언락한다. 데드락은 회피하는게 답이다. 

In [77]:
import threading
import time

class ThreadPool(object):
    def __init__(self): #생성자에서 락킹 지원
        self.active = []
        self.lock = threading.Lock() 
        
    def acquire(self, name): #락을 획득하는 것은 락을 거는 것.
        with self.lock: # 이것을 오토락이라고 함 맨처음 진입하는 애에 대해 락을 획득
            self.active.append(name)
            print("획득: {0} | 스레드풀: {1}".format(name, self.active))
            
    def release(self, name): #락을 풀때 지원.
        with self.lock: 
            self.active.remove(name)
            print("반환: {0} | 스레드풀: {1}".format(name, self.active))         
            
def worker(semaphore, pool):
    with semaphore: #여기서 세마포어의
        name = threading.currentThread().getName() #임계영역으로 3개만 들어가는 것, 들어온 이름 뽑아내기 위해 겟네임
        pool.acquire(name) # 임계영역에 들어가서
        time.sleep(1)# 1초 있다가 임계영역에서
        pool.release(name) #이러면 빠져나온다
            
if __name__ == "__main__":
    
    threads = []
    pool = ThreadPool()
    semaphore = threading.Semaphore(3) #임계영역에 3개 들어갈 수 있게 설정
    for i in range(10):
        t = threading.Thread( target=worker, name="스레드 "+str(i), args=(semaphore,pool))#세마포어와 스레드 풀을 만들어서 넘겨주는것) # 스레딩 만드는 것
        t.start() # 기동 시키고
        threads.append(t) #
        
    for t in threads:
        t.join() #모든 스레드가 종료할 때 까지 메인스레드는 대기탄다

획득: 스레드 0 | 스레드풀: ['스레드 0']
획득: 스레드 1 | 스레드풀: ['스레드 0', '스레드 1']
획득: 스레드 2 | 스레드풀: ['스레드 0', '스레드 1', '스레드 2']
반환: 스레드 0 | 스레드풀: ['스레드 1', '스레드 2']
획득: 스레드 3 | 스레드풀: ['스레드 1', '스레드 2', '스레드 3']
반환: 스레드 1 | 스레드풀: ['스레드 2', '스레드 3']
반환: 스레드 2 | 스레드풀: ['스레드 3']
획득: 스레드 4 | 스레드풀: ['스레드 3', '스레드 4']
획득: 스레드 5 | 스레드풀: ['스레드 3', '스레드 4', '스레드 5']
반환: 스레드 3 | 스레드풀: ['스레드 4', '스레드 5']
획득: 스레드 6 | 스레드풀: ['스레드 4', '스레드 5', '스레드 6']
반환: 스레드 4 | 스레드풀: ['스레드 5', '스레드 6']
획득: 스레드 7 | 스레드풀: ['스레드 5', '스레드 6', '스레드 7']
반환: 스레드 5 | 스레드풀: ['스레드 6', '스레드 7']
획득: 스레드 8 | 스레드풀: ['스레드 6', '스레드 7', '스레드 8']
반환: 스레드 6 | 스레드풀: ['스레드 7', '스레드 8']
획득: 스레드 9 | 스레드풀: ['스레드 7', '스레드 8', '스레드 9']
반환: 스레드 7 | 스레드풀: ['스레드 8', '스레드 9']
반환: 스레드 8 | 스레드풀: ['스레드 9']
반환: 스레드 9 | 스레드풀: []


### 파이썬 멀티 스레드와 멀티 프로세스
> 파이썬은 인터프리터 언어로서 기본적으로 싱글 스레드에서 순차적으로 동작한다. 따라서 병렬처리를 하기 위해서는 별도의 모듈을 사용하여 구현해야함. threading모듈을 이용한 스레드 구현과 multiprocessing모듈을 이용한 프로세스 구현을 보고자 함

### threading 모듈

* 파이썬에서 쓰레드를 실행하기 위해서는 threading 모듈의 threading.Thread()함수를 호출하여 Thread객체를 얻은 후 Thread객체의 start()메서드를 호출하면 된다. subthread는 함수 혹은 메서드를 실행하는데, 일반적인 구현방식으로 1)쓰레드가 실행할 함수 혹은 메서드를 작성하거나 또는 2)threading.Thread로부터 파생된 파생클래스를 작성하여 사용하는 방식등이 있다.

    - 1) 함수 및 메서드 실행 방식은 스레드가 실행할 함수(혹은 메서드)를 작성하고 그 함수명을 threading.Thread()함수의 target arguments 에 지정하면 된다. 만약 스레드가 실행하는 함수(혹은 메서드)에 입력 parameter를 전달해야 한다면, args(혹은 kwargs)에 필요한 parameter를 지정하면 된다. args는 튜플로 파라미터를 전달하고 kwargs는 dict로 전달한다. 

### 생산자 소비자의 문제

* 데드락(교착상태) deadlock은 두 개 이상의 프로세스나 스레드가 서로 상대방의 작업이 끝나기만을 기다리고 있기 때문에 결과적으로 아무것도 완료되지 못하는 상태다
    - 프로그램에서 락을 할당하고, 락을 순서대로 획득한다면 교착 상태를 막을 수 있다
    
* 스핀락 spinlock은 고성능 컴퓨팅 상황에 유용한 busy waiting의 한 형태다
    - 스핀락은 Critical Section에 진입이 불가능할 때, 진입이 가능할 때까지 반복문을 돌면서 재시도 하는 방식으로 구현된 락이다
    
    
* queue모듈의 Queue데이터 타입을 스레드 간 데이터를 전달하는 기본 방식으로 사용한다. 
    - 그렇지 않으면 threading모듈의 락을 사용한다. threading.Condition을 사용할 수 있도록 조건변수를 적절하게 사용하는 방법을 숙지한다. 조건변수를 사용하면 스레드가 순번을 제어할 수 있다.  아래 예제는 순번제어에 목적이 있다.

In [8]:
import threading

def consumer(cond):
    name = threading.currentThread().getName() 
    print("{0} 시작".format(name))#시작과 대기와 소비에 대한 print
    with cond:
        print("{0} 대기".format(name))
        cond.wait() # 생산된게 없으면 여기서 대기하게 되고 깨어나면
        print("{0} 자원 소비".format(name))
        
def producer(cond):
    name = threading.currentThread().getName() 
    print("{0} 시작".format(name))#시작과 대기와 소비에 대한 print
    with cond:
        print("{0} 자원 생산 후 모든 소비자에게 알림".format(name))
        cond.notifyAll() #대기타고 있는 모든 스레드에게 생산을 알림


if __name__ == "__main__":
    
    condition = threading.Condition() #컨디션은 인자가 없다
    consumer1 = threading.Thread( target=consumer, name="소비자1", args=(condition,))#args는 동기화 제어
    consumer2 = threading.Thread( target=consumer, name="소비자2", args=(condition,))
    producer = threading.Thread( target=producer, name="생산자", args=(condition,)) #소비자와의 통신 목적으로 사용된다
    
    consumer1.start() #시작해도 대기타게 됨
    consumer2.start() # 마찬가지다
    producer.start() #시작해서 생산하면 기동한다 
    

소비자1 시작
소비자1 대기
소비자2 시작
소비자2 대기
생산자 시작
생산자 자원 생산 후 모든 소비자에게 알림
소비자1 자원 소비
소비자2 자원 소비


###  프로파일러

In [14]:
import cProfile
import time

def sleep():
    time.sleep(5)

def hello_world():
    print("Hello world!")
    
def main():
    sleep()
    hello_world()
    
cProfile.run('main()')

Hello world!
Hello world!
         70 function calls in 3.009 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.009    3.009 <ipython-input-14-ec60c9bc125a>:10(main)
        1    0.000    0.000    3.009    3.009 <ipython-input-14-ec60c9bc125a>:4(sleep)
        2    0.000    0.000    0.000    0.000 <ipython-input-14-ec60c9bc125a>:7(hello_world)
        1    0.000    0.000    3.009    3.009 <string>:1(<module>)
        5    0.000    0.000    0.000    0.000 iostream.py:197(schedule)
        4    0.000    0.000    0.000    0.000 iostream.py:309(_is_master_process)
        4    0.000    0.000    0.000    0.000 iostream.py:322(_schedule_flush)
        4    0.000    0.000    0.000    0.000 iostream.py:384(write)
        5    0.000    0.000    0.000    0.000 iostream.py:93(_event_pipe)
        5    0.000    0.000    0.000    0.000 socket.py:342(send)
        5    0.000    0.000    0.000    0.000 threa

In [15]:
import cProfile
import time

def sleep():
    time.sleep(5)

def hello_world():
    print("Hello world!")
    
def main():
    sleep()
    hello_world()
    hello_world() #ncalls가 2가 된다. 호출이 두번 되는 거라서
    
cProfile.run('main()')

Hello world!
Hello world!
         70 function calls in 5.014 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    5.014    5.014 <ipython-input-15-1fb640dec76d>:10(main)
        1    0.000    0.000    5.014    5.014 <ipython-input-15-1fb640dec76d>:4(sleep)
        2    0.000    0.000    0.000    0.000 <ipython-input-15-1fb640dec76d>:7(hello_world)
        1    0.000    0.000    5.014    5.014 <string>:1(<module>)
        5    0.000    0.000    0.000    0.000 iostream.py:197(schedule)
        4    0.000    0.000    0.000    0.000 iostream.py:309(_is_master_process)
        4    0.000    0.000    0.000    0.000 iostream.py:322(_schedule_flush)
        4    0.000    0.000    0.000    0.000 iostream.py:384(write)
        5    0.000    0.000    0.000    0.000 iostream.py:93(_event_pipe)
        5    0.000    0.000    0.000    0.000 socket.py:342(send)
        5    0.000    0.000    0.000    0.000 threa

### 스택 만들기 

In [20]:
class Stack(object):
    def __init__(self):
        self.items = []
        
    def isEmpty(self):
        return not bool(self.items) #리스트가 비어있을 때만 True리턴
    
    def push(self, value):
        self.items.append(value)
        
    def pop(self): #인자를 뽑지 않아도 리턴만 하면 됨
        value = self.items.pop()
        if value is not None: #데이터가 있다면
            return value
        else: #비어있는 경우
            print("Stack is empty")
            
    def size(self):
        return len(self.items) #items의 원소의 갯수 센다
    
    def peek(self):
        if self.items:
            return self.items[-1]
        else:
            print("Stack is empty.")
            
    def __repr__(self):
        return repr(self.items)
    
if __name__ == "__main__":
    stack = Stack()
    print(stack) #데이터 넣기 전
    for i in range(10):
        stack.push(i)
    print(stack) #넣은 후 , 가장 밑에 0이 있게 됨
    print("pop : {0}".format(stack.pop())) #마지막 숫자인 9, 최종숫자라 뽑혀나옴
    print("pop : {0}".format(stack.pop())) #스텍이구나!
    

[]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
pop : 9
pop : 8


In [21]:
class Stack(object):
    def __init__(self):
        self.items = []
        
    def isEmpty(self):
        return not bool(self.items) #리스트가 비어있을 때만 True리턴
    
    def push(self, value):
        self.items.append(value)
        
    def pop(self): #인자를 뽑지 않아도 리턴만 하면 됨
        value = self.items.pop()
        if value is not None: #데이터가 있다면
            return value
        else: #비어있는 경우
            print("Stack is empty")
            
    def size(self):
        return len(self.items) #items의 원소의 갯수 센다
    
    def peek(self):
        if self.items:
            return self.items[-1]
        else:
            print("Stack is empty.")
            
    def __repr__(self):
        return repr(self.items)
    
if __name__ == "__main__":
    stack = Stack()
    print(stack) #데이터 넣기 전
    for i in range(10):
        stack.push(i)
    print(stack) #넣은 후 , 가장 밑에 0이 있게 됨
    for i in range(10):
        print("pop : {0}".format(stack.pop()))
    print("pop : {0}".format(stack.pop())) # 데이터 익셉션이 나오게 된다. 
    

[]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
pop : 9
pop : 8
pop : 7
pop : 6
pop : 5
pop : 4
pop : 3
pop : 2
pop : 1
pop : 0


IndexError: pop from empty list

In [25]:
class Node(object): # 노드가 되는 것
    def __init__(self, value=None, pointer=None ): # 원래 파이썬에는 pointer란게 없다
        self.value = value
        self.pointer = pointer 

class Stack(object):
    def __init__(self):
        self.head = None
        self.count = 0 # 원소의 수 관리
        
    def isEmpty(self):
        return not bool(self.head) #리스트가 비어있을 때만 True리턴
    
    def push(self, item): #어느 방향으로든 노드를 추가해준다
        self.head = Node(item, self.head) # insert code이다.
        self.count += 1
        
    def pop(self): #인자를 뽑지 않아도 리턴만 하면 됨
        if(self.count > 0 and self. head):
            node = self.head # 맨 앞에서 추출하는 코드
            self.head = node.pointer # 중간 노드를 없앤다는 것
            self.count -= 1 
            return node.value #노드가 있다면 노드 안에 있는 것을 리턴
        else : # 노드가 없다면
            print("Stack is empty")
            
    def size(self):
        return self.count
    
    def peek(self):
        if(self.count > 0 and self. head): #데이터가 있다면 데이터를 지우지 않고 그 값을 리턴
            return self.node.value # 헤드의 value 리턴
        else:
            print("Stack is empty.")
            
    def _printList(self):
        node = self.head
        while node:
            print(node.value, end=" ")
            node = node.pointer
        print()
    
if __name__ == "__main__":
    stack = Stack()
    print(stack) #데이터 넣기 전
    for i in range(10):
        stack.push(i)
    print(stack) #넣은 후 , 가장 밑에 0이 있게 됨
    for i in range(10):
        print("pop : {0}".format(stack.pop()))
    print("pop : {0}".format(stack.pop())) # 데이터 익셉션이 나오게 된다. 

pop : 9
pop : 8
pop : 7
pop : 6
pop : 5
pop : 4
pop : 3
pop : 2
pop : 1
pop : 0
Stack is empty
pop : None


### 큐(queue) 

In [27]:
#Stack 리스트 구현 코드를 가져와서 약간 변형

class Queue(object):
    def __init__(self):
        self.items = []
        
    def isEmpty(self):
        return not bool(self.items) #리스트가 비어있을 때만 True리턴
    
    def enqueue(self, item): #0인덱스 앞에 넣는 것은 맨 앞에 넣는다는 것.
        self.items.insert(0, item) #insert는 앞에서 넣는것. append는 맨뒤에 넣는거고.
        
    def dequeue(self): #인자를 뽑지 않아도 리턴만 하면 됨
        value = self.items.pop()
        if value is not None: #데이터가 있다면
            return value
        else: #비어있는 경우
            print("Queue is empty")
            
    def size(self):
        return len(self.items) #items의 원소의 갯수 센다
    
    def peek(self):
        if self.items:
            return self.items[-1]
        else:
            print("Queue is empty.")
            
    def __repr__(self):
        return repr(self.items)
    
if __name__ == "__main__":
    queue = Queue()
    print(queue) #데이터 넣기 전
    for i in range(10):
        queue.enqueue(i)
    print(queue) #넣은 후 , 가장 밑에 0이 있게 됨
    for i in range(10):
        print("dequeue : {0}".format(queue.dequeue()))

        #집어넣은 순서대로 뽑으면 이렇게 나온다. 

[]
[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
dequeue : 0
dequeue : 1
dequeue : 2
dequeue : 3
dequeue : 4
dequeue : 5
dequeue : 6
dequeue : 7
dequeue : 8
dequeue : 9


In [31]:
#Stack 의 노드 타입에서 좀 바뀌는 식

class Node(object): # 노드가 되는 것
    def __init__(self, value=None, pointer=None ): # 원래 파이썬에는 pointer란게 없다
        self.value = value
        self.pointer = pointer 

class LinkedQueue(object):
    def __init__(self): #맨앞, 맨뒤 가리키는 노드 만든다
        self.head = None
        self.tail = None
        self.count = 0 # 원소의 수 관리
        
    def isEmpty(self):
        return not bool(self.head) #리스트가 비어있을 때만 True리턴
    
    def dequeue(self): 
        if(self.head ): #벨류를 뽑아서 리턴
            value = self.head.value
            self.head = self.head.pointer # 맨 앞에서 지운다는 것
            self.count -= 1 
            return node.value 
        else : # 노드가 없다면
            print("Queue is empty")
    
    def enqueue(self, value): 
        node = Node(value) # 바로 헤드에 넣지 않았다
        # 테일 쪽에 넣는다!
        if not self.head: #헤드가 none이라면
            self.head = node
            self.tail = node #그때는 노드가 하나뿐이라서 tail도 노드 가리키면 됨
        else:
            if self.tail: #테일이 none이 아니라면
                self.tail.pointer = node
            self.tail = node # 이게 맨뒤에 넣는 코드다
        self.count += 1 # 노드가 들어왓으니까 +1 하는 것이다
            
    def size(self):
        return self.count
    
    def peek(self):
        if(self.count > 0 ): 
            return self.head.value # 헤드의 value 리턴
        else:
            print("Queue is empty.")
            
    def _printList(self):
        node = self.head
        while node:
            print(node.value, end=" ")
            node = node.pointer
        print()
    
if __name__ == "__main__":
    queue = Queue()
    print(queue) #데이터 넣기 전
    for i in range(10):
        queue.enqueue(i)
    print(queue) #넣은 후 , 가장 밑에 0이 있게 됨
    for i in range(10):
        print("dequeue : {0}".format(queue.dequeue()))
    print("dequeue : {0}".format(queue.dequeue())) # 데이터 익셉션이 나오게 된다. 

[]
[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
dequeue : 0
dequeue : 1
dequeue : 2
dequeue : 3
dequeue : 4
dequeue : 5
dequeue : 6
dequeue : 7
dequeue : 8
dequeue : 9


IndexError: pop from empty list

###  heapq 모듈

In [32]:
import heapq
list1 = [4,6,8,1]
heapq.heapify(list1)
list1

[1, 4, 8, 6]

In [37]:
h = []
heapq.heappush( h, (1, 'food'))
heapq.heappush( h, (2, 'have fun'))
heapq.heappush( h, (3, 'work'))
heapq.heappush( h, (4, 'study'))
type(h)
h


[(1, 'food'), (2, 'have fun'), (3, 'work'), (4, 'study')]

In [38]:
list1

[1, 4, 8, 6]

In [39]:
heapq.heappop(list1) #가장 작은 값이 뽑혀 나오는 느낌

1

In [40]:
heapq.heappop(list1)

4

In [41]:
heapq.heappop(list1)

6

In [42]:
heapq.heappop(list1) # 들어간 순서는 1 4 8 6 인데 뽑혀나오는 순서는 1 4 6 8

8

### heap 구현


In [72]:
class Heapify(object):
    def __init__(self, data=None):
        self.data = data or []
        for i in range(len(data)//2, -1, -1): # range(start , end , step)
            #print(i)
            self.__max_heapify__(i)
            
    def parent(self, i):
        if i & 1: # 가장 끝에 있는 i를 비교해서 홀수인지 아닌지를 비교한 것이다.
            return i >> 1  #i 가 인덱스이므로 오른쪽으로 1비트 미는 것은 2로 나누는 것이다.
        else: # 짝수인 경우 부모를 찾을 때 I를 오른쪽으로 1칸 밀어 2로 나누는 식
            return (i >> 1) -1
        
            
    def left_child(self, i):
        return(i<<1) + 1 # 정수를 1비트만큼 왼쪽으로 밀라는 것은 2을 곱하라는 것
    # I<<1은 곱하기 2하는 거라고 보면 된다.
    
    def right_child(self, i):
        return (i<<1) + 2
    
    def __max_heapify__(self, i):  # 업힙과 다운 힙
        largest = i 
        left = self.left_child(i) #여기가 3이 나온다
        right = self.right_child(i)
        n = len(self.data) #전체 원소 수를 말한다. 
        
        largest = (left<n and self.data[left] > self.data[i]) and left or i #범위가 넘어가는 것은 무조건 false리턴해서 계산 안함
        largest = (right<n and self.data[right] > self.data[largest]) and right or largest
        
        if i is not largest:
            self.data[i] , self.data[largest] = self.data[largest], self.data[i] # tuple unpacking
            print(self.data)
            self.__max_heapify__(largest) #재귀
            
    def extract_max(self):
        n = len(self.data)
        max_element = self.data[0]
        self.data[0] = self.data[n-1] # 가장 끝에있는 원소를 내림
        self.data = self.data[:n-1]
        self.__max_heapify__(0)
        
        return max_element #이러고나면 8이 나오고 그다음 큰 값이 맨 위로 올라간다.
        
    def insert(self, item):
        i = len(self.data)
        self.data.append(item)
        while ( i!=0) and item > self.data[self.parent(i)]:
#             print(self.data)
            self.data[i] = self.data[self.parent(i)]
            i = self.parent(i)
        self.data[i]=item
        
def test_heapify():
    l1 = [3,2,5,1,7,8,2]
    h = Heapify(l1) # h 가 생성됨
    print(h.extract_max()) #여기서 무조건 8이 나옴 
    print(h.extract_max())
    print(h.extract_max())
    h.insert(9)
    print(h.data) # insert 9가 들어가있다.
    

if __name__ == "__main__":
    test_heapify()
    

[3, 2, 8, 1, 7, 5, 2]
[3, 7, 8, 1, 2, 5, 2]
[8, 7, 3, 1, 2, 5, 2]
[8, 7, 5, 1, 2, 3, 2]
[7, 2, 5, 1, 2, 3]
8
[5, 2, 3, 1, 2]
7
[3, 2, 2, 1]
5
[9, 3, 2, 1, 2]


In [73]:
class Heapify(object):
    def __init__(self, data=None):
        self.data = data or []
        for i in range(len(data)//2, -1, -1): # range(start , end , step)
            #print(i)
            self.__max_heapify__(i)
            
    def parent(self, i):
        if i & 1: # 가장 끝에 있는 i를 비교해서 홀수인지 아닌지를 비교한 것이다.
            return i >> 1  #i 가 인덱스이므로 오른쪽으로 1비트 미는 것은 2로 나누는 것이다.
        else: # 짝수인 경우 부모를 찾을 때 I를 오른쪽으로 1칸 밀어 2로 나누는 식
            return (i >> 1) -1
        
            
    def left_child(self, i):
        return(i<<1) + 1 # 정수를 1비트만큼 왼쪽으로 밀라는 것은 2을 곱하라는 것
    # I<<1은 곱하기 2하는 거라고 보면 된다.
    
    def right_child(self, i):
        return (i<<1) + 2
    
    def __max_heapify__(self, i):  #  다운 힙
        largest = i 
        left = self.left_child(i) #여기가 3이 나온다
        right = self.right_child(i)
        n = len(self.data) #전체 원소 수를 말한다. 
        
        
        largest = (left<n and self.data[left] > self.data[i]) and left or i #범위가 넘어가는 것은 무조건 false리턴해서 계산 안함
        # 이게 무엇을 하는 것인가?
        # 
        largest = (right<n and self.data[right] > self.data[largest]) and right or largest # 부모랑 왼쪽자식이랑 라지스트랑 이렇게 세개를 비교하게 됨
        #결국 나와 왼쪽과 오른쪽을 비교해 라지스트에 값을 넣어라 이것
        
        #가장큰 값이 i라면 이 아래의 if를 돌지 않는다. 바꾼것에 대해 다시 재귀호출로 가장 큰값의 index기준으로 돌아간다.
        if i is not largest:   #오로지 조건에 맞춰서 크고 작은것을 스왑하는 것, 즉 리스트의 순번만을 바꿔주는것
            self.data[i] , self.data[largest] = self.data[largest], self.data[i] # tuple unpacking
            print(self.data)
            self.__max_heapify__(largest) #재귀
            
    def extract_max(self): # 다운 힙 동작
        n = len(self.data)
        max_element = self.data[0]
        self.data[0] = self.data[n-1] # 가장 끝에있는 원소를 내림
        self.data = self.data[:n-1]
        self.__max_heapify__(0)
        
        return max_element #이러고나면 8이 나오고 그다음 큰 값이 맨 위로 올라간다.
        
    def insert(self, item):#업힙 동작
        i = len(self.data)
        self.data.append(item)
        while ( i!=0) and item > self.data[self.parent(i)]:
#             print(self.data)
            self.data[i] = self.data[self.parent(i)]
            i = self.parent(i)
        self.data[i]=item
        
def test_heapify():
    l1 = [3,2,5,1,7,8,2]
    h = Heapify(l1) # h 가 생성됨
    print(h.extract_max()) #여기서 무조건 8이 나옴 
    print(h.extract_max())
    print(h.extract_max())
    h.insert(9)
    print(h.data) # insert 9가 들어가있다.
    

if __name__ == "__main__":
    test_heapify()
    

[3, 2, 8, 1, 7, 5, 2]
[3, 7, 8, 1, 2, 5, 2]
[8, 7, 3, 1, 2, 5, 2]
[8, 7, 5, 1, 2, 3, 2]
[7, 2, 5, 1, 2, 3]
8
[5, 2, 3, 1, 2]
7
[3, 2, 2, 1]
5
[9, 3, 2, 1, 2]


In [75]:
class Heapify(object):
    def __init__(self, data=None):
        self.data = data or []
        for i in range(len(data)//2, -1, -1): # range(start , end , step)
            #print(i)
            self.__max_heapify__(i)
            
    def parent(self, i):
        if i & 1: # 가장 끝에 있는 i를 비교해서 홀수인지 아닌지를 비교한 것이다.
            return i >> 1  #i 가 인덱스이므로 오른쪽으로 1비트 미는 것은 2로 나누는 것이다.
        else: # 짝수인 경우 부모를 찾을 때 I를 오른쪽으로 1칸 밀어 2로 나누는 식
            return (i >> 1) -1
        
            
    def left_child(self, i):
        return(i<<1) + 1 # 정수를 1비트만큼 왼쪽으로 밀라는 것은 2을 곱하라는 것
    # I<<1은 곱하기 2하는 거라고 보면 된다.
    
    def right_child(self, i):
        return (i<<1) + 2
    
    def __max_heapify__(self, i):  
        largest = i 
        left = self.left_child(i) #여기가 3이 나온다
        right = self.right_child(i)
        n = len(self.data) #전체 원소 수를 말한다. 
        
        
        largest = (left<n and self.data[left] > self.data[i]) and left or i #범위가 넘어가는 것은 무조건 false리턴해서 계산 안함
        # 이게 무엇을 하는 것인가?
        # 
        largest = (right<n and self.data[right] > self.data[largest]) and right or largest # 부모랑 왼쪽자식이랑 라지스트랑 이렇게 세개를 비교하게 됨
        #결국 나와 왼쪽과 오른쪽을 비교해 라지스트에 값을 넣어라 이것
        
        #가장큰 값이 i라면 이 아래의 if를 돌지 않는다. 바꾼것에 대해 다시 재귀호출로 가장 큰값의 index기준으로 돌아간다.
        if i is not largest:   #오로지 조건에 맞춰서 크고 작은것을 스왑하는 것, 즉 리스트의 순번만을 바꿔주는것
            self.data[i] , self.data[largest] = self.data[largest], self.data[i] # tuple unpacking
            print(self.data)
            self.__max_heapify__(largest) #재귀
            
    def extract_max(self): # 다운 힙동작
        n = len(self.data)
        max_element = self.data[0]
        self.data[0] = self.data[n-1] # 가장 끝에있는 원소를 내림
        self.data = self.data[:n-1]
        self.__max_heapify__(0)
        
        return max_element #이러고나면 8이 나오고 그다음 큰 값이 맨 위로 올라간다.
        
    def insert(self, item):#업힙 동작
        i = len(self.data)
        self.data.append(item)
        while ( i!=0) and item > self.data[self.parent(i)]:
#             print(self.data)
            self.data[i] = self.data[self.parent(i)]
            i = self.parent(i)
        self.data[i]=item
        
def test_heapify():
    l1 = [1,2,3,4,5,6,7]
    h = Heapify(l1) # h 가 생성됨
    print(h.extract_max()) #여기서 무조건 8이 나옴 
    print(h.extract_max())
    print(h.extract_max())
    h.insert(9)
    print(h.data) # insert 9가 들어가있다.
    

if __name__ == "__main__":
    test_heapify()
    

[1, 2, 7, 4, 5, 6, 3]
[1, 5, 7, 4, 2, 6, 3]
[7, 5, 1, 4, 2, 6, 3]
[7, 5, 6, 4, 2, 1, 3]
[6, 5, 3, 4, 2, 1]
7
[5, 1, 3, 4, 2]
[5, 4, 3, 1, 2]
6
[4, 2, 3, 1]
5
[9, 4, 3, 1, 2]
